# Musk

[`Musk dataset`](https://archive.ics.uci.edu/ml/datasets/Musk+\(Version+2\)) is a classic MIL problem of the field, introduced in [Thomas G. Dietterich, Richard H. Lathrop, Tomás Lozano-Pérez (1997)](http://www.sciencedirect.com/science/article/pii/S0004370296000343). Below we demonstrate how to solve this problem using [`Mill.jl`](https://github.com/CTUAvastLab/Mill.jl).
The full environment and the script is accessible [here](https://github.com/CTUAvastLab/Mill.jl/tree/master/docs/src/examples/musk).

We start by activating the environment and installing required packages

In [1]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/work/Mill.jl/Mill.jl/docs/src/examples/musk`
Status `~/work/Mill.jl/Mill.jl/docs/src/examples/musk/Project.toml`
  [5789e2e9] FileIO v1.16.0
  [587475ba] Flux v0.13.14
  [033835bb] JLD2 v0.4.31
  [1d0525e4] Mill v2.8.3 `../../../..`
  [e88e6eb3] Zygote v0.6.59
  [10745b16] Statistics


We load all dependencies and fix the seed:

In [2]:
using FileIO, JLD2, Statistics, Mill, Flux
using Flux: throttle, @epochs
using Mill: reflectinmodel
using Base.Iterators: repeated

using Random; Random.seed!(42);

Then we load the dataset and transform it into a `Mill` structure. The `musk.jld2` file contains...
* a matrix with features, each column is one instance:

In [3]:
fMat = load("musk.jld2", "fMat")

166×476 Matrix{Float32}:
   42.0    42.0    42.0    42.0    42.0  …    38.0    43.0    39.0    52.0
 -198.0  -191.0  -191.0  -198.0  -198.0     -123.0  -102.0   -58.0  -121.0
 -109.0  -142.0  -142.0  -110.0  -102.0     -139.0   -20.0    27.0   -24.0
  -75.0   -65.0   -75.0   -65.0   -75.0       30.0  -101.0    31.0  -104.0
 -117.0  -117.0  -117.0  -117.0  -117.0     -117.0  -116.0  -117.0  -116.0
   11.0    55.0    11.0    55.0    10.0  …   -88.0   200.0   -92.0   195.0
   23.0    49.0    49.0    23.0    24.0      214.0  -166.0    85.0  -162.0
  -88.0  -170.0  -161.0   -95.0   -87.0      -13.0    66.0    21.0    76.0
  -28.0   -45.0   -45.0   -28.0   -28.0      -74.0  -222.0   -73.0  -226.0
  -27.0     5.0   -28.0     5.0   -28.0     -129.0   -49.0   -68.0   -56.0
    ⋮                                    ⋱                             ⋮
  -74.0  -302.0   -73.0  -302.0   -73.0     -226.0    32.0  -232.0    34.0
 -129.0    60.0  -127.0    60.0  -127.0     -210.0   136.0  -206.0   133.0
 -

* the ids of samples (*bags* in MIL terminology) specifying to which each instance (column in `fMat`) belongs to:

In [4]:
bagids = load("musk.jld2", "bagids")

476-element Vector{Int64}:
  1
  1
  1
  1
  2
  2
  2
  2
  3
  3
  ⋮
 91
 92
 92
 92
 92
 92
 92
 92
 92

* and labels defined on the level of instances:

In [5]:
y = load("musk.jld2", "y")

476-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

We create a `BagNode` structure which holds:
1. feature matrix and
2. ranges identifying which columns in the feature matrix each bag spans.

In [6]:
ds = BagNode(ArrayNode(fMat), bagids)

BagNode  # 92 obs, 1.500 KiB
  ╰── ArrayNode(166×476 Array with Float32 elements)  # 476 obs, 308.703 KiB

This representation ensures that feed-forward networks do not need to deal with bag boundaries and always process full continuous matrices:

We also compute labels on the level of bags. In the `Musk` problem, bag label is defined as a maximum of instance labels (i.e. a bag is positive if at least one of its instances is positive):

In [7]:
y = map(i -> maximum(y[i]) + 1, ds.bags)
y_oh = Flux.onehotbatch(y, 1:2)

2×92 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

Once the data are in `Mill` internal format, we will manually create a model. `BagModel` is designed to implement a basic multi-instance learning model utilizing two feed-forward networks with an aggregaton operator in between:

In [8]:
model = BagModel(
    Dense(166, 10, Flux.tanh),
    BagCount(SegmentedMeanMax(10)),
    Chain(Dense(21, 10, Flux.tanh), Dense(10, 2)))

BagModel ↦ BagCount([SegmentedMean(10); SegmentedMax(10)]) ↦ Chain(Dense(21 => ⋯
  ╰── ArrayModel(Dense(166 => 10, tanh))  # 2 arrays, 1_670 params, 6.602 KiB

Instances are first passed through a single layer with 10 neurons (input dimension is 166) with `tanh` non-linearity, then we use `mean` and `max` aggregation functions simultaneously (for some problems, max is better then mean, therefore we use both), and then we use one layer with 10 neurons and `tanh` nonlinearity followed by linear layer with 2 neurons (output dimension). We check that forward pass works

In [9]:
model(ds)

2×92 Matrix{Float32}:
 1.00559   0.71191   1.14493   1.19852   …  0.665943  0.016557  0.477948
 0.800909  0.486771  0.790684  0.714799     0.45441   0.295348  1.42942

Since `Mill` is entirely compatible with [`Flux.jl`](https://fluxml.ai), we can use its `cross-entropy` loss function:...

In [10]:
loss(ds, y_oh) = Flux.logitcrossentropy(model(ds), y_oh)

loss (generic function with 1 method)

...and run simple training procedure using its tooling:

In [11]:
opt = Flux.ADAM()
@epochs 10 begin
    Flux.train!(loss, Flux.params(model), repeated((ds, y_oh), 1000), opt)
    println(loss(ds, y_oh))
end

┌ Warning: The macro `@epochs` will be removed from Flux 0.14.
│ As an alternative, you can write a simple `for i in 1:epochs` loop.
│   caller = eval at boot.jl:368 [inlined]
└ @ Core ./boot.jl:368
[ Info: Epoch 1
0.018946962
[ Info: Epoch 2
0.0037844975
[ Info: Epoch 3
0.0013878477
[ Info: Epoch 4
0.00063628616
[ Info: Epoch 5
0.00032373675
[ Info: Epoch 6
0.00017440894
[ Info: Epoch 7
9.721945e-5
[ Info: Epoch 8
5.539199e-5
[ Info: Epoch 9
3.2017975e-5
[ Info: Epoch 10
1.8711045e-5


Finally, we calculate the (training) error:

In [12]:
mean(mapslices(argmax, model(ds), dims = 1)' .≠ y)

0.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*